In [5]:
import numpy as np
import matplotlib.pyplot as plt
from pycbc.waveform import get_td_waveform
from pycbc.detector import Detector
from pycbc.coordinates import spherical_to_cartesian
from ipywidgets import Checkbox, IntSlider, FloatSlider, Layout, HBox, VBox, interactive_output

h1 = Checkbox(value=True, description='LIGO H1')
l1 = Checkbox(value=False, description='LIGO L1')
v1 = Checkbox(value=False, description='Virgo')
k1 = Checkbox(value=False, description='KAGRA')

mass1 = IntSlider(min=3, max=50, step=1, value=30, description="Mass1")
mass2 = IntSlider(min=3, max=50, step=1, value=30, description="Mass2")
distance = IntSlider(min=500, max=1500, step=1, value=500, description="Distance")
dec = FloatSlider(min=0, max=np.pi, step=0.01, value=0, description="Dec")
ra = FloatSlider(min=0, max=2*np.pi, step=0.01, value=0, description="RA")
inclination = FloatSlider(min=0, max=2*np.pi, step=0.01, value=0, description="Inclination")
pol = FloatSlider(min=0, max=2*np.pi, step=0.01, value=0, description="Polar angle")
spin1a = FloatSlider(min=0, max=0.99, step=0.01, value=0, description="spin1_amp")
spin1az = FloatSlider(min=0, max=2*np.pi, step=0.01, value=0, description="spin1_az")
spin1po = FloatSlider(min=0, max=np.pi, step=0.01, value=0, description="spin1_polar")
spin2a = FloatSlider(min=0, max=0.99, step=0.01, value=0, description="spin2_amp")
spin2az = FloatSlider(min=0, max=2*np.pi, step=0.01, value=0, description="spin2_az")
spin2po = FloatSlider(min=0, max=np.pi, step=0.01, value=0, description="spin2_polar")
coa_phase = FloatSlider(min=0, max=2*np.pi, step=0.01, value=0, description="Coa phase")

def plot_gw(mass1, mass2, distance, dec, ra, inclination, pol, coa_phase, spin1a, spin1az, spin1po, spin2a, spin2az, spin2po, h1, l1, v1, k1):
    spin1x, spin1y, spin1z = spherical_to_cartesian(spin1a, spin1az, spin1po)
    spin2x, spin2y, spin2z = spherical_to_cartesian(spin2a, spin2az, spin2po)
    hp, hc = get_td_waveform(approximant="IMRPhenomXPHM", mass1=mass1, mass2=mass2, distance=distance, coa_phase=coa_phase, inclination=inclination,
                            spin1x=spin1x, spin1y=spin1y, spin1z=spin1z, spin2x=spin2x, spin2y=spin2y, spin2z=spin2z, delta_t=1.0/2048, f_lower=20)
    
    plotDetector = (h1,l1,v1,k1)
    waves = {}
    for i, det in enumerate(('H1', 'L1', 'V1', 'K1')):
        if plotDetector[i]:
            wave = Detector(det).project_wave(hp, hc, ra, dec, pol)
            waves[det] = wave.time_slice(float(wave.end_time)-3, float(wave.end_time))
            
    plt.figure(figsize=(20,8))
    for det in waves:
        plt.plot(waves[det].sample_times, waves[det], label=det)
    plt.xlabel('Time from merger(s)', fontsize=16)
    plt.ylabel('Strain', fontsize=16)
    plt.ylim(-2e-21, 2e-21)
    if 1 in plotDetector:
        plt.legend(fontsize=16)
    plt.grid()

out = interactive_output(plot_gw,
                         {'mass1':mass1, 'mass2':mass2, 'distance':distance, 'dec':dec, 'ra':ra, 'pol':pol, 'coa_phase':coa_phase, 'inclination':inclination, 
                          'spin1a':spin1a, 'spin1az':spin1az, 'spin1po':spin1po, 'spin2a':spin2a, 'spin2az':spin2az, 'spin2po':spin2po,
                          'h1':h1, 'l1':l1, 'v1':v1, 'k1':k1})
check_boxes = HBox([h1, l1, v1, k1])
sliders = VBox([HBox([mass1, mass2, distance, coa_phase]), HBox([spin1a, spin1az, spin1po]), HBox([spin2a, spin2az, spin2po]), HBox([dec, ra, inclination,pol])])
display(check_boxes, sliders, out)
print('注) 質量の単位は太陽質量、距離の単位はMpcである。')

Output()

注) 質量の単位は太陽質量、距離の単位はMpcである。
